# Weight regression

### Libraries and Variables

In [2]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as nnf
import numpy as np
import os
from torchinfo import summary
from glob import glob
import pandas as pd
import numpy as np
import os
import random
from PIL import Image
import torchvision.transforms as transforms
import logging
import os
import time
from torch import distributed as dist
from torchvision.models import resnet34, ResNet34_Weights
from torch import nn
import torch.nn.functional as F
import numpy as np
from torch import nn
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import os.path as os.path
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

from exlib.model import WeightResNet34
from exlib.dataset import WeightDataset
from torch.utils.data import DataLoader

home_dir = os.path.expanduser('~')
raw_data_dir = os.path.join(home_dir, 'repos/DaNuMa2024/data/raw_data')
output_data_dir = os.path.join(home_dir, 'repos/DaNuMa2024/data/output_data')

### Overview

In this notebook, you will enhance the MLP architecture from the last exercise with a well-known regularization technique, namely "dropout". Furthermore, you will implement a convolutional neural network and demonstrate its superiority over the MLP when it comes to image processing.

### Data exploration

In [ ]:
# tabular data
train_weights_path = os.path.join(raw_data_dir, '5_weight_regression/train.csv')
val_weights_path = os.path.join(raw_data_dir, '5_weight_regression/val.csv')
train_weights = pd.read_csv(train_weights_path)
val_weights = pd.read_csv(val_weights_path)

# images directory
images_dir = os.path.join(raw_data_dir, '5_weight_regression/images')

In [38]:
# explore csv data
print(train_weights.head())
print('\n')
print(f'number of training examples: {train_weights.shape[0]}')
print(f'number of validation examples: {val_weights.shape[0]}')
print(f'mean weight: {train_weights.weight.mean()}')
print(f'std weight: {train_weights.weight.std()}')
print(f'min weight: {train_weights.weight.min()}')
print(f'max weight: {train_weights.weight.max()}')

   weight                       images_dir
0    52.5  Gr_2_WG_2_900222000834743_depth
1    37.0  Gr_2_WG_2_900222000834745_depth
2    41.5  Gr_2_WG_2_900222000834748_depth
3    34.0  Gr_2_WG_2_900222000834749_depth
4    49.0  Gr_2_WG_2_900222000834750_depth


number of training examples: 347
number of validation examples: 148
mean weight: 54.68155619596542
std weight: 13.573825326246093
min weight: 27.0
max weight: 102.5


In [ ]:
# plot frames of one pig
index = 5
images_one_pig_dir = os.path.join(images_dir, train_weights['images_dir'][index])
images_one_pig = os.listdir(images_one_pig_dir)
images_one_pig = sorted(images_one_pig, key=lambda x: int(x[:-4].split('_')[-1]))

n_rows = 3
n_cols = 10
figsize = (20, 5)
fig, axs = plt.subplots(n_rows, n_cols, figsize=figsize)
axs = axs.ravel()

for i in range(n_rows * n_cols):
    if i < len(images_one_pig):
        img_path = os.path.join(images_one_pig_dir, images_one_pig[i])
        img = mpimg.imread(img_path)
        axs[i].imshow(img)
        axs[i].axis('off') 
    else:
        axs[i].axis('off')

plt.show()

### Dataset

In [ ]:
class WeightDataset(Dataset):
    def __init__(self, weights_df_path, images_base_dir):
        self.weights_df = pd.read_csv(weights_df_path)
        self.images_base_dir = images_base_dir
        self.transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomVerticalFlip(p=0.5),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])
                                ])

    def __getitem__(self, i):
        # select row from dataframe and get data from it
        info = self.weights_df.iloc[i, :]
        weight = torch.tensor(info.weight).float()
        images_folder = info.images_dir

        # load one random image corresponding to the weighting of the selected row
        images_dir = os.path.join(self.images_base_dir, images_folder)
        image_name = random.choice(os.listdir(images_dir))
        image_path = os.path.join(images_dir, image_name)
        image = Image.open(image_path)

        # transform and return image
        image = self.transform(image)
        return image, weight

    def __len__(self):
        return len(self.weights_df)

In [ ]:
# def is_main_process():
#     rank, _ = get_dist_info()
#     return rank == 0


# # RETURNS UNIQUE IDENTIFIER OF THE CURRENT PROCESS WITHIN DISTRIBUTED PROCESS GROUP (rank) AND THE NUMBER OF SUCH PROCESSES - 1 (world_size)
# def get_dist_info():
#     if dist.is_available() and dist.is_initialized():
#         rank = dist.get_rank()
#         world_size = dist.get_world_size()
#     else:
#         rank = 0
#         world_size = 1
#     return rank, world_size


# def get_root_logger(log_file=None, log_level=logging.INFO):
#     logger = logging.getLogger('acoustics')
#     # if the logger has been initialized, just return it
#     if logger.hasHandlers():
#         return logger
#     logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=log_level)
#     if not is_main_process():
#         logger.setLevel('ERROR')
#     elif log_file is not None:
#         file_handler = logging.FileHandler(log_file, 'w')
#         file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
#         file_handler.setLevel(log_level)
#         logger.addHandler(file_handler)

#     return logger


# def init_train_logger(save_directory, config=None):
#     # Make sure previous handlers are removed
#     for handler in logging.root.handlers[:]:
#         logging.root.removeHandler(handler)

#     os.makedirs(os.path.abspath(save_directory), exist_ok=True)
#     timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
#     log_file = os.path.join(save_directory, f'{timestamp}.log')
#     logger = get_root_logger(log_file=log_file)
#     # logger.info(f'Config:\n{args}')
#     # shutil.copy(args.config, os.path.join(save_directory, os.path.basename(args.config)))
#     return logger


# def print_log(msg, logger=None, level=logging.INFO):
#     """Print a log message.
#     Args:
#         msg (str): The message to be logged.
#         logger (logging.Logger | str | None): The logger to be used.
#             Some special loggers are:
#             - "silent": no message will be printed.
#             - other str: the logger obtained with `get_root_logger(logger)`.
#             - None: The `print()` method will be used to print log messages.
#         level (int): Logging level. Only available when `logger` is a Logger
#             object or "root".
#     """
#     if logger is None:
#         print(msg)
#     elif isinstance(logger, logging.Logger):
#         logger.log(level, msg)
#     elif logger == 'silent':
#         pass
#     else:
#         raise TypeError(
#             'logger should be either a logging.Logger object, '
#             f'"silent" or None, but got {type(logger)}')


# def close_logger(logger):
#     handlers = logger.handlers[:]
#     for handler in handlers:
#         logger.removeHandler(handler)
#         handler.close()


# def back_to_original(image):
#     image = image * torch.tensor([0.229, 0.224, 0.225])[:, None, None]
#     image = image + torch.tensor([0.485, 0.456, 0.406])[:, None, None]
#     image = image * 255
#     image = image.numpy()
#     image = image.astype('uint8')
#     image = np.transpose(image, [1, 2, 0])
#     return image
    

### Model

In [ ]:
class WeightCnn(nn.Module):
    def __init__(self):
        super().__init__()
        ######### YOUR CODE HERE:
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(32)
        self.bn5 = nn.BatchNorm2d(64)
        self.bn6 = nn.BatchNorm2d(64)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 1)
        self.fc.bias.data.fill_(50) # important

    def forward(self, x):
        ######### YOUR CODE HERE:
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.avg_pool(x).squeeze()
        x = self.fc(x)
        return x

class WeightResNet34(nn.Module):
    def __init__(self):
        ######### YOUR CODE HERE:
        super().__init__()
        self.resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
        self.resnet34.fc = nn.Linear(512, 1)
        self.resnet34.fc.bias.data.fill_(50)

    def forward(self, x):
        x = self.resnet34(x)
        return x

### training loop

In [ ]:
def train_one_epoch(model, trainloader, optimizer, device):
    ######### YOUR CODE HERE:
    model.train()
    total_loss = 0
    for x_batch, y_batch in trainloader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(x_batch)

        optimizer.zero_grad()
        loss = nnf.mse_loss(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(trainloader)


def validate(model, valloader, device):
    ######### YOUR CODE HERE:
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_batch, y_batch in valloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x_batch)
            loss = nnf.mse_loss(y_pred, y_batch)
            total_loss += loss.item()
    return total_loss / len(valloader)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"The model is running on {device}.")

# training parameters
epochs = 150
lr = 0.001
val_interval = 1
batch_size = 8
decay_factor = 0.1
patience = 20
print_interval = 5

# save best model state dict and metrics
save_dir_state_dict = os.path.join(output_data_dir, '5_weight_regression')
os.makedirs(save_dir_state_dict, exist_ok=True)
save_path_state_dict = os.path.join(save_dir_state_dict, 'best.pth')
save_path_metrics = os.path.join(save_dir_state_dict, 'metrics.pkl')

# instantiate dataset and dataloader
train_weights_path = os.path.join(raw_data_dir, '5_weight_regression/train.csv')
val_weights_path = os.path.join(raw_data_dir, '5_weight_regression/val.csv')
images_dir = os.path.join(raw_data_dir, '5_weight_regression/images')
trainset = WeightDataset(train_weights_path, images_dir)
valset = WeightDataset(val_weights_path, images_dir)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False)    

# Initialize model, optimizer and scheduler
model = WeightResNet34().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=decay_factor, patience=patience)

# train loop
train_losses = []
val_losses = []
min_val_loss = float('inf')
for epoch in tqdm(range(epochs)):
    train_loss = train_one_epoch(model, optimizer, trainloader, device)
    train_losses.append(train_loss)
    scheduler.step(val_loss)
    
    if epoch % val_interval == 0:
        val_loss = validate(model, valloader, device)
        val_losses.append(val_loss)
        
    if val_loss < min_val_loss:
        torch.save(model.state_dict(), save_path_state_dict)
        min_val_loss = val_loss
        
    if epoch % print_interval == 0:
        print(f'Epoch {epoch} - train loss: {train_loss:.3f} - val loss: {val_loss:.3f}')
    
    metrics = pd.DataFrame({
        'train_loss': train_losses,
        'val_loss': val_losses,
        'lr': optimizer.param_groups[0]['lr']
    })
    metrics.to_pickle(save_path_metrics)

### results

In [ ]:
results = pd.read_pickle(save_path_metrics)

In [ ]:
# plt.ylim([0,200])

####################### plot losses
plt.plot(np.linspace(1, epochs, epochs), results['train_loss'], c='blue', label='Training Loss')
plt.plot(np.linspace(1, epochs, epochs), results['val_loss'], c='red', label='Validation Loss')

# Mark the minimum validation loss
index = np.argmin(val_losses)
plt.plot(index+1, val_losses[index], 'kx', label='Min Validation Loss')

# Adding labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
val_weights_path = os.path.join(raw_data_dir, '5_weight_regression/val.csv')
# val_weights = pd.read_csv(val_weights_path)
images_dir = os.path.join(raw_data_dir, '5_weight_regression/images')

# dataset and trained model
valset = WeightDataset(val_weights_path, images_dir)
valloader = DataLoader(valset, batch_size=1)
model = WeightResNet34()
best_ckpt = torch.load('results/best.pth')
model.load_state_dict(best_ckpt)
model = model.to(device)
model.eval()

In [ ]:
preds = []
targets = []
with torch.no_grad():
    for image, target in tqdm(valloader):
        image = image.to(device)
        pred = model(image)
        preds.append(pred.item())
        targets.append(target.item())

In [ ]:
plt.scatter(targets, preds)